In [1]:
import numpy as np
import pandas as pd
import pickle
import csv
import time as tm
import sys
import functools
print = functools.partial(print, flush=True)

In [2]:
start_time = []
def tic():
    start_time.append(tm.time())

def toc():
    print("--- %s seconds ---" % (tm.time() - start_time.pop()))

In [3]:
def safeDiv(dividend,divisor) :
    return dividend/divisor if divisor != 0 else 0

no_data = 0
def probability(distribution, indices, default=-1) :
    if len(indices) == 1 :
        if (isinstance(distribution,dict) and indices[0] not in distribution) or \
        (distribution is None) or \
        (distribution[indices[0]] == 0 and distribution[0] == 0) :
            global no_data
            no_data += 1
            return default
        else :
            return safeDiv(distribution[indices[0]], distribution[0])
    return probability(distribution[indices[0]],indices[1:], default)

# def tallyUp(array, indices, weight=1, verbose=False) :
#     if len(indices) == 1 :
#         array[0] += weight
#         if indices[0] != 0 :
#             array[indices[0]] += weight
#         if verbose :
#             print(array)
#     else :
#         tallyUp(array[0], indices[1:], weight, verbose)
#         if indices[0] != 0 :
#             tallyUp(array[indices[0]], indices[1:], weight, verbose)



In [4]:
# Unpickle
tic()
prediction_and_print = ['events_lookup','category_list','topic_list','US_region_code','CA_region_code','top25_code','top25','advertisers_dict']+['states','provinces','countries']+['topics','categories','days','advertisers']+['campaigns']

for each in prediction_and_print :
    %time globals()[each] = pickle.load(open('bayes_model\\model_builder\\{}.p'.format(each),'rb'))
    print('{:20s}:    {:10,d} bytes'.format(each,globals()[each].nbytes if isinstance(globals()[each],np.ndarray) else sys.getsizeof(globals()[each])))

toc()

Wall time: 1.72 s
events_lookup       :    554,883,024 bytes
Wall time: 8.19 s
category_list       :    200,277,112 bytes
Wall time: 10.6 s
topic_list          :    200,277,112 bytes
Wall time: 10.5 ms
US_region_code      :         3,168 bytes
Wall time: 5 ms
CA_region_code      :           864 bytes
Wall time: 5 ms
top25_code          :         1,632 bytes
Wall time: 11 ms
top25               :           200 bytes
Wall time: 15 ms
advertisers_dict    :     2,292,396 bytes
Wall time: 8.01 ms
states              :           672 bytes
Wall time: 7.01 ms
provinces           :           232 bytes
Wall time: 10 ms
countries           :         2,392 bytes
Wall time: 8.01 ms
topics              :         2,400 bytes
Wall time: 6 ms
categories          :           776 bytes
Wall time: 9.01 ms
days                :           144 bytes
Wall time: 6.01 ms
advertisers         :        35,080 bytes
Wall time: 15 ms
campaigns           :     2,292,396 bytes
--- 20.717214345932007 seconds ---


In [5]:
tic()
supplies = ['loc','day','time','plat','cat','top','advsr']+['advmt','camp']

for each in supplies :
    with open('bayes_model\\{}3.p'.format(each),'rb') as reader :
        %time globals()[each] = pickle.load(reader)
    print('{:20s}:    {:10,d} bytes'.format(each,globals()[each].nbytes if isinstance(globals()[each],np.ndarray) else sys.getsizeof(globals()[each])))

with open('bayes_model\\clicks.p','rb') as reader :
    %time clicks = pickle.load(reader)

toc()

Wall time: 0 ns
loc                 :           576 bytes
Wall time: 0 ns
day                 :            96 bytes
Wall time: 1 ms
time                :         2,400 bytes
Wall time: 0 ns
plat                :         9,600 bytes
Wall time: 352 ms
cat                 :    90,316,800 bytes
Wall time: 1 ms
top                 :       707,952 bytes
Wall time: 5.01 ms
advsr               :     2,526,336 bytes
Wall time: 882 ms
advmt               :           112 bytes
Wall time: 2.14 s
camp                :           112 bytes
Wall time: 0 ns
--- 3.4841885566711426 seconds ---


In [101]:
%time clicks_test = pd.read_csv('clicks_test/clicks_test.csv', dtype={'display_id':np.int,'ad_id':np.int}, na_filter=False)
clicks_test.head(7)

Wall time: 7.66 s


,display_id,ad_id
0,16874594,66758
1,16874594,150083
2,16874594,162754
3,16874594,170392
4,16874594,172888
5,16874594,180797
6,16874595,8846


In [102]:
display_id = clicks_test['display_id'].tolist()
ad_id = clicks_test['ad_id'].tolist()

%xdel clicks_test

In [103]:
# predictions
tic()
t0 = tm.time()
# document_id, platform, state, country, hour, weekday, category_pairs, topic_pairs
# US_region_code, CA_region_code, top25_code, top25, advertisers_dict
# probability, safeDiv
# display_id, ad_id, clicked
# loc, day, time, plat, cat, top, advsr
# topics, categories, days

predictions = []

# Track how much missing data
no_data = 0
no_cat = 0
no_top = 0
default = 1#probability(clicks, [1+1])

mean_topic_1 = [np.mean([probability(top, 
                                     [1+1,1+category,1+topic],default=default)  # remember that probability() defaults negative
                         for topic in range(len(topics))])
                for category in range(len(categories))]
mean_topic_0 = [np.mean([probability(top, 
                                     [1+0,1+category,1+topic],default=default)  # remember that probability() defaults negative
                         for topic in range(len(topics))])
                for category in range(len(categories))]
mean_topic = [np.mean([probability(top, 
                                   [0,1+category,1+topic],default=default)
                       for topic in range(len(topics))])
              for category in range(len(categories))]

display = display_id[0]
offset = display_id[0]
previous = -1

location = 0

index = 0

def get_cat_top_probability(click,location,weekday,hour,platform,category_pairs,topic_pairs) :
    global no_cat, no_top, default
    exit = []
    flags = 0
    cat_top_probability = 0
    if category_pairs is not None : # have a category label
        cat_prob = 0
        cat_base = 0
        for category, confidence in category_pairs :
            inc = confidence * safeDiv(probability(cat, [1+click,1+location,1+weekday,1+hour,1+platform-1,1+category]),
                                       probability(cat, [0,1+location,1+weekday,1+hour,1+platform-1,1+category]))
            if inc < 0 :
                inc = 1#confidence * default
                flags += 1
            
            cat_prob = inc

            if topic_pairs is not None : # have a topic label
                top_prob = 0
                top_base = 0
                for topic, tconfidence in topic_pairs :
                    inc = tconfidence * safeDiv(probability(top, [1+click,1+category,1+topic]),
                                                probability(top, [0,1+category,1+topic]))  # this was previously always 1... lol

                    if inc < 0 :
                        inc = 0#tconfidence * default
                        flags += 1
                    
                    top_prob += inc
                    
                    top_base += tconfidence

                top_prob = safeDiv(top_prob, top_base)
                cat_top_probability += cat_prob * top_prob
                cat_base += confidence
                exit.append(1)

            else : # no topic label, default to average topic probability
                no_top += 2828649/5481475 # weighted for repetitons
                cat_top_probability += cat_prob * safeDiv(mean_topic_1[category] if click == 1 else mean_topic_0[category],
                                                          mean_topic[category])
                cat_base += confidence
                exit.append(2)

        cat_top_probability = safeDiv(cat_top_probability, cat_base)

    else : #no category label, default to average category probability
        no_cat += 1
        cat_prob = safeDiv(np.mean([probability(cat,
                                                [1+click,1+location,1+weekday,1+hour,1+platform-1,1+c],default=default)
                                    for c in range(len(categories))]),
                           np.mean([probability(cat, 
                                                [0,1+location,1+weekday,1+hour,1+platform-1,1+c],default=default) 
                                    for c in range(len(categories))]))

        if cat_prob < 0 :
            cat_prob = 1#default
            flags += 1

        if topic_pairs is not None : # have a topic label
            top_prob = 0
            top_base = 0
            for topic, tconfidence in topic_pairs :
                inc = tconfidence * safeDiv(probability(top, [1+click,0,1+topic]),
                                            probability(top, [0,0,1+topic])) # this was previously always 1... lol

                if inc < 0 :
                    inc = 1#tconfidence * default
                    flags += 1

                top_prob += inc
                
                top_base += tconfidence

            top_prob = safeDiv(top_prob, top_base)
            cat_top_probability += cat_prob * top_prob
            exit.append(3)

        else : # no topic label, default to some number
            no_top += 1 # no repetitons
            cat_top_probability += cat_prob * default
            exit.append(4)
    
    return (flags, cat_top_probability, exit)

def joint(click=-1,location=-1,weekday=-1,hour=-1,platform=-1,category=-1,topic=-1,advertiser=-1,campaign=-1,advertisement=-1) :
    global clicks, loc, day, time, plat, cat, top, advsr, camp, advmt, default
    result = 1
    if click != -1 :
        result *= probability(clicks, [1+click])
    if location != -1 :
        result *= probability(loc, [1+click,1+location], default=default)
    if weekday != -1 :
        result *= probability(day, [1+click,1+weekday], default=default)
    if hour != -1 :
        result *= probability(time, [1+click,1+weekday,1+hour], default=default)
    if platform != -1 :
        result *= probability(plat, [1+click,1+weekday,1+hour,1+platform-1], default=default)
    if category != -1 :
        result *= probability(cat, [1+click,1+location,1+weekday,1+hour,1+platform-1,1+category], default=default)
    if topic != -1 :
        result *= probability(top, [1+click,1+category,1+topic],default=default)
    if advertiser != -1 :
        result *= probability(advsr, [1+click,1+location,1+advertiser],default=default)
    if campaign != -1 :
        result *= probability(camp, [1+click,1+location,1+advertiser,campaign],default=default) # Tricky! campaign was indexed inconsistently in training.. sigh
    if advertisement != -1 :
        result *= probability(advmt, [1+click,1+campaign,1+advertisement],default=default) # campaign is indexed conventionally again here.. should retrain with better indices..
    
    return result


for display, ad in zip(display_id,ad_id) :
    
    # Do this once per new display_id group
    if previous != display :
        
        document, platform, state, country, hour, weekday, category_pairs, topic_pairs = [events_lookup[0][display - offset],
                                                                                          events_lookup[1][display - offset],
                                                                                          events_lookup[2][display - offset],
                                                                                          events_lookup[3][display - offset],
                                                                                          events_lookup[4][display - offset],
                                                                                          events_lookup[5][display - offset],
                                                                                          category_list[display - offset],
                                                                                          topic_list[display - offset]
                                                                                         ]
        
        if country == 237 :
            if state != -1 :
                location = 25+US_region_code[state]
            else :
                location = 25+14 # default to Northeast US
        elif country == 41 :
            if state != -1 :
                location = 25+19+CA_region_code[state]
            else :
                location = 25+19+1 # default to Southeast CA
        else :
            if country in top25 :
                location = top25_code[country]
            else :
                location = 25+14 # default to Northeast US
        
        cat_top_1 = get_cat_top_probability(1,location,weekday,hour,platform,category_pairs,topic_pairs)
        cat_top_2 = get_cat_top_probability(0,location,weekday,hour,platform,category_pairs,topic_pairs)
        
        N1 = (loc[1+1][1+location] 
              + day[1+1][1+weekday]
              + time[1+1][1+weekday][1+hour]
              + plat[1+1][1+weekday][1+hour][1+platform-1]
              #+ cat[1+1][1+location][1+weekday][1+hour][1+platform-1][0]
              #+ top[1+1][1+category][0]
             )
        N2 = (loc[1+0][1+location] 
              + day[1+0][1+weekday]
              + time[1+0][1+weekday][1+hour]
              + plat[1+0][1+weekday][1+hour][1+platform-1]
              #+ cat[1+0][1+location][1+weekday][1+hour][1+platform-1][0]
              #+ top[1+0][1+category][0]
             )

        p1 = safeDiv(cat_top_1[1]*joint(1,location,weekday,hour,platform),joint(-1,location,weekday,hour,platform))
        p2 = safeDiv(cat_top_2[1]*joint(0,location,weekday,hour,platform),joint(-1,location,weekday,hour,platform))
        P = (p1, p2)
        
        previous = display
        predictions.append([])
    
    # Do this every iteration
    adv = advertisers_dict[ad]
    campaign = campaigns[ad]
    
    flags = [cat_top_1[0], cat_top_2[0]]
    denom = (probability(advsr, [0,1+location,1+adv], default=default)
             *probability(camp, [0,1+location,1+adv,campaign], default=default)
             *probability(advmt, [0,1+campaign,ad], default=default))
    
    p1 = probability(advsr, [1+1,1+location,1+adv])
    p2 = probability(camp, [1+1,1+location,1+adv,campaign])
    p3 = probability(advmt, [1+1,1+campaign,ad])
             
    if p1 == -1 :
        p1 = probability(advsr, [0,1+location,1+adv], default=default)
        flags[0] += 1
    if p2 == -1 :
        p2 = probability(camp, [0,1+location,1+adv,campaign], default=default)
        flags[0] += 1
    if p3 == -1 :
        p3 = probability(advmt, [0,1+campaign,ad], default=default)
        flags[0] += 1
    
    cp1 = probability(advsr, [1+0,1+location,1+adv])
    cp2 = probability(camp, [1+0,1+location,1+adv,campaign])
    cp3 = probability(advmt, [1+0,1+campaign,ad])
    
    if cp1 == -1 :
        cp1 = probability(advsr, [0,1+location,1+adv], default=default)   # Forgot to standardize to these to [0-1] after leaving out the default to detect error above!
        flags[1] += 1
    if cp2 == -1 :
        cp2 = probability(camp, [0,1+location,1+adv,campaign], default=default)
        flags[1] += 1
    if cp3 == -1 :
        cp3 = probability(advmt, [0,1+campaign,ad], default=default)
        flags[1] += 1
    
    
    #flags[0] = -99
    if flags[0] < flags[1] :
        _P = safeDiv(P[0]*p1*p2*p3,denom)
    elif flags[0] > flags[1] :
        _P = 1 - safeDiv(P[1]*cp1*cp2*cp3,denom)   # Almost forgot the "1 - " part! Also the "c" in front of p1,p2,p3!
    else :
        _N1 = N1
        try :
            _N1 += advsr[1+1][1+location][1+adv]
        except TypeError :
            pass
        except KeyError :
            pass
        try :
            _N1 += camp[1+1][1+location][1+adv][campaign]
        except TypeError :
            pass
        except KeyError :
            pass
        try :
            _N1 += advmt[1+1][1+campaign][ad]
        except TypeError :
            pass
        except KeyError :
            pass
        
        _N2 = N2
        try :
            _N2 += advsr[1+0][1+location][1+adv]
        except TypeError :
            pass
        except KeyError :
            pass
        try :
            _N2 += camp[1+0][1+location][1+adv][campaign]
        except TypeError :
            pass
        except KeyError :
            pass
        try :
            _N2 += advmt[1+0][1+campaign][ad]
        except TypeError :
            pass
        except KeyError :
            pass

        _P = _N1*safeDiv(P[0]*p1*p2*p3, denom) + _N2*(1 - safeDiv(P[1]*cp1*cp2*cp3, denom))  # parenthesis around "(1-...)" matter!
        _P /= (_N1 + _N2)
    
#     #assert cat_top_2[1]-1.048965666850497 < .00000001 and
#     assert True or index in {168,198,199,203,233} or _P >= 0 and _P <= 1, '\n'.join(['{}: Probability out of bounds (<0 or >1)'.format(index),
#                                          'P \t\t= {:.6f}'.format(_P),
#                                          'P[0] \t\t= {:.6f}'.format(P[0]),
#                                          'P[0]*p1*p2*p3/denom \t= {:.6f}'.format(safeDiv(P[0]*p1*p2*p3,denom)),
#                                          '',
#                                          'P[1] \t\t= {:.6f}'.format(P[1]),
#                                          '1-P[1]*cp1*cp2*cp3/denom = {:.6f}'.format(1-safeDiv(P[1]*cp1*cp2*cp3,denom)),
#                                          '',
#                                          'denom \t\t= {:.15f}'.format(denom),
#                                          'cat_top1: \t({},{})'.format(cat_top_1[0],cat_top_1[1]),
#                                          'cat_top2: \t({},{})'.format(cat_top_2[0],cat_top_2[1]),
#                                          'p1 = \t\t{:.6f}'.format(p1),
#                                          'p2 = \t\t{:.6f}'.format(p2),
#                                          'p3 = \t\t{:.6f}'.format(p3),
#                                          'cp1 = \t\t{:.6f}'.format(cp1),
#                                          'cp2 = \t\t{:.6f}'.format(cp2),
#                                          'cp3 = \t\t{:.6f}'.format(cp3),
#                                          '_N1 = \t\t{:d}'.format(_N1),
#                                          '_N2 = \t\t{:d}'.format(_N2),
#                                          'flags[0] = \t\t{:d}'.format(flags[0]),
#                                          'flags[1] = \t\t{:d}'.format(flags[1]),
#                                          ', '.join([str(i) for i in cat_top_1[2]]),
#                                          ', '.join([str(i) for i in cat_top_2[2]])
#                                         ])
    
    predictions[display - offset].append((ad,_P))

    # Print progress
    if index % int(len(display_id)/100) == 0 :
        print(predictions[display-offset][-1],end='\t\t')
        print('Time since start: {:.3f} sec'.format(tm.time()-t0))
        print('{percent:5.1f}% {index:3d}e5    P({cond}|X)={p1:05.3f}'.format(
                percent=index/len(display_id)*100,
                index=int(index/100000),
                p1=predictions[display-offset][-1][1],
                cond=1
            )
        )
        #if input('continue? (y/n)') == 'n' :
        #    break

    index += 1

toc()

(66758, 0.050274104915412003)		Time since start: 0.378 sec
  0.0%   0e5    P(1|X)=0.050
(127047, 0.43735406458245674)		Time since start: 45.269 sec
  1.0%   3e5    P(1|X)=0.437
(261042, 0.048035081024430056)		Time since start: 91.167 sec
  2.0%   6e5    P(1|X)=0.048
(288364, 0.045456236548844849)		Time since start: 136.858 sec
  3.0%   9e5    P(1|X)=0.045
(362034, 0.25294640213606712)		Time since start: 182.038 sec
  4.0%  12e5    P(1|X)=0.253
(184220, 0.41822973855726936)		Time since start: 286.477 sec
  5.0%  16e5    P(1|X)=0.418
(206997, 0.21134890234769793)		Time since start: 385.116 sec
  6.0%  19e5    P(1|X)=0.211
(37095, 0.066642437804955337)		Time since start: 433.869 sec
  7.0%  22e5    P(1|X)=0.067
(378025, 0.20392030868040401)		Time since start: 485.757 sec
  8.0%  25e5    P(1|X)=0.204
(173387, 0.092259109196715816)		Time since start: 529.340 sec
  9.0%  29e5    P(1|X)=0.092
(132815, 0.22638335276464444)		Time since start: 569.086 sec
 10.0%  32e5    P(1|X)=0.226
(209278, 0.

In [95]:
clicks_train = pd.read_csv('clicks_train\\clicks_train.csv',dtype={'display_id':int,'ad_id':int,'clicked':int},nrows=1000000,skiprows=2435244,names=['display_id','ad_id','clicked'],na_filter=False)
clicks_train.head(10)

,display_id,ad_id,clicked
0,475103,12845,0
1,475103,191468,0
2,475103,275253,1
3,475104,74987,0
4,475104,155839,0
5,475104,207649,1
6,475104,303479,0
7,475105,83251,0
8,475105,141393,0
9,475105,183230,1


In [90]:
clicks_train.tail(10)

,display_id,ad_id,clicked
999990,665989,202311,1
999991,665989,331468,0
999992,665990,26711,0
999993,665990,129628,0
999994,665990,190062,1
999995,665990,266938,0
999996,665990,281764,0
999997,665990,288402,0
999998,665991,99763,0
999999,665991,141393,0


In [96]:
display_id = clicks_train.loc[3:999997,'display_id'].tolist()
ad_id = clicks_train.loc[3:999997,'ad_id'].tolist()
clicked = clicks_train.loc[3:999997,'clicked'].tolist()

In [92]:
%xdel clicks_train

In [98]:
def score(row) :#, least_to_greatest=False) :
    for i,(ad_id,probability,clicked) in enumerate(row) :
        if clicked == 1 :
            #if least_to_greatest :
            #    return 1/(len(row)-i)
            #else :
            return 1/(i+1)

tic()
scores = np.zeros((len(predictions,)))

for i,display_row in enumerate(predictions) :
    display_row.sort(key=lambda x: x[1], reverse=True)
    scores[i] = score(display_row)
    #scores[i] = score(np.sort(np.array(display_row, dtype=[('ad',int),('prob',float),('clicked',int)]),order='prob'),
    #                  least_to_greatest=True)
    if i % int(len(predictions)/100) == 0 :
        print('%d Score: %f' % (i, scores[i]))

toc()
print('Score: %f' % np.mean(scores))

0 Score: 0.250000
1908 Score: 0.333333
3816 Score: 0.500000
5724 Score: 1.000000
7632 Score: 1.000000
9540 Score: 1.000000
11448 Score: 0.500000
13356 Score: 1.000000
15264 Score: 1.000000
17172 Score: 0.200000
19080 Score: 1.000000
20988 Score: 0.333333
22896 Score: 1.000000
24804 Score: 1.000000
26712 Score: 1.000000
28620 Score: 1.000000
30528 Score: 0.500000
32436 Score: 0.333333
34344 Score: 0.333333
36252 Score: 0.500000
38160 Score: 0.333333
40068 Score: 1.000000
41976 Score: 1.000000
43884 Score: 0.333333
45792 Score: 1.000000
47700 Score: 1.000000
49608 Score: 0.333333
51516 Score: 1.000000
53424 Score: 1.000000
55332 Score: 1.000000
57240 Score: 0.500000
59148 Score: 0.333333
61056 Score: 0.500000
62964 Score: 1.000000
64872 Score: 0.500000
66780 Score: 0.333333
68688 Score: 1.000000
70596 Score: 1.000000
72504 Score: 0.500000
74412 Score: 0.250000
76320 Score: 0.333333
78228 Score: 0.333333
80136 Score: 1.000000
82044 Score: 0.333333
83952 Score: 1.000000
85860 Score: 0.3333

In [99]:
no_data, no_cat, no_top, default

(61279, 22336, 12830.8856222097, 1)

In [104]:
tic()
with open('bayes_model\\predictions\\submission1.1_11.28.csv', 'w', newline='') as csvfile:
    filewriter = csv.writer(csvfile, delimiter=' ',
                            quotechar='|', quoting=csv.QUOTE_MINIMAL)
    filewriter.writerow(['display_id,ad_id'])
    for display, disp_group in zip(np.unique(display_id), predictions) :
        disp_group.sort(key=lambda x: x[1], reverse=True)
        filewriter.writerow(['{},{}'.format(display,disp_group[0][0])] + [ad for (ad,prob) in disp_group[1:]])

toc()

--- 118.38253140449524 seconds ---


In [16]:
assert False, "Hi"

AssertionError: Hi

In [108]:
invalids = [1 if prob < 0 or prob > 1 else 0 for disp in predictions for (ad,prob) in disp]
sum(invalids), len(display_id)

(538847, 32225162)

In [ ]:
(48e5,366324,.7565), (154e5, 468636,-2.005), (219e5,553996,-2.025), (251e5,550729, 0), (254e5,282555,20.160), (309e5, 565599,-2.211)

In [107]:
514802/ 32225162

0.0159751563079807

In [109]:
538847 /  32225162

0.01672131237074929

In [110]:
no_data, no_cat, no_top

(22361334, 577668, 656204.9213601394)

In [13]:
ad_id[4800000-2:4800000+2]

[91681, 139376, 141295, 232122]

In [14]:
int(len(display_id)/100)

322251

In [15]:
48e5/322251

14.895221426776022

In [16]:
15*322251

4833765

In [17]:
ad_id[4833765-2:4833765+2]

[150799, 187290, 366324, 413233]

In [19]:
display_id[4833765], ad_id[4833765]

(17810617, 366324)

In [20]:
display_id[4833765-2:4833765+2]

[17810617, 17810617, 17810617, 17810617]

In [21]:
 document, platform, state, country, hour, weekday, category_pairs, topic_pairs = [events_lookup[0][display - offset],
                                                                                          events_lookup[1][display - offset],
                                                                                          events_lookup[2][display - offset],
                                                                                          events_lookup[3][display - offset],
                                                                                          events_lookup[4][display - offset],
                                                                                          events_lookup[5][display - offset],
                                                                                          category_list[display - offset],
                                                                                          topic_list[display - offset]
                                                                                         ]

In [22]:
document, platform, state, country, hour, weekday, category_pairs, topic_pairs

(1373663,
 3,
 9,
 237,
 17,
 2,
 None,
 [(83, 0.0190337569453875),
  (127, 0.0094897491308752503),
  (192, 0.0090774146809802208),
  (47, 0.0089925566234316297)])

26

In [25]:

cat_top_1 = get_cat_top_probability(1,location,weekday,hour,platform)
cat_top_1
       

(0, 1.0)

In [ ]:
cat_top_2 = get_cat_top_probability(0,location,weekday,hour,platform)